In [92]:
from os import listdir
from os.path import isfile, join
from spacy.lang.pl import Polish
from spacy.tokenizer import Tokenizer
from math import log

In [93]:
tokenizer = Tokenizer(Polish().vocab)

data_path = '../dane'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [94]:
#zad 2
single_grams = {}
bigrams = {}
trigrams = {}

for filename in files:
    file = None
    with open(data_path +'/' + filename, 'r', encoding = 'utf-8') as f:
        file = f.read().lower()

    tokens = tokenizer(file)
    
    for i in range(len(tokens)-1):
        token1 = tokens[i].text
        token2 = tokens[i + 1].text
        
        single_grams[token1] = single_grams.get(token1, 0) + 1
        bigram = (token1, token2)
        bigrams[bigram] = bigrams.get(bigram, 0) + 1
        
       
    for i in range(len(tokens)-2):
        token1 = tokens[i].text
        token2 = tokens[i + 1].text
        token3 = tokens[i + 2].text
        trigram = (token1, token2, token3)

        trigrams[trigram] = trigrams.get(trigram, 0) + 1
      
    last_token = tokens[len(tokens)-1].text
    single_grams[last_token] = single_grams.get(last_token, 0) + 1
    
len(bigrams)

1097691

In [95]:
#zad3
polish_letters = 'aąbcćdeęfghijklłmnńoópqrsśtuvwyxzźż'
def only_polish_letters(word):
    if len(word) == 0:
        return False
    for l in word:
        if not l in polish_letters:
            return False
    return True

# odfiltrownaie 
single_grams = {k: v for k,v in single_grams.items() if only_polish_letters(k)}
bigrams = {k: v for k,v in bigrams.items() if only_polish_letters(k[0]) and  only_polish_letters(k[1])}
print(len(bigrams))

440588


In [116]:
#zad 4
total_single_occurances = sum(single_grams.values())
total_bigrams_occurances = sum(bigrams.values())
bigram_pmi = {}

for bigram, freq in bigrams.items():
    t1, t2 = bigram
    p1 = single_grams[t1]/total_single_occurances
    p2 = single_grams[t2]/total_single_occurances
    p1p2 = freq/total_bigrams_occurances
    pmi = log(p1p2/(p1 * p2))
    bigram_pmi[bigram] = (freq, pmi)

In [117]:
#zad 5
bigram_pmi = {k: v for k,v in sorted(bigram_pmi.items(), key=lambda item: item[1][1], reverse=True)}
bigram_top_10_pmi = list(bigram_pmi.keys())[:10]
list(bigram_pmi.items())[:10]

[(('kołowe', 'jednoosiowe'), (1, 15.232429554616882)),
 (('automatyki', 'grzewczej'), (1, 15.232429554616882)),
 (('prefabrykatów', 'wnętrzowe'), (1, 15.232429554616882)),
 (('gołe', 'aluminiowe'), (1, 15.232429554616882)),
 (('polistyrenu', 'spienionego'), (1, 15.232429554616882)),
 (('objaśnieniem', 'figur'), (1, 15.232429554616882)),
 (('wkładzie', 'wnoszonym'), (1, 15.232429554616882)),
 (('doktorem', 'habilitowanym'), (1, 15.232429554616882)),
 (('naruszonymi', 'plombami'), (1, 15.232429554616882)),
 (('zaniedbanej', 'wychowawczo'), (1, 15.232429554616882))]

In [118]:
# zad 6
bigram_pmi_filtered = {k: v for k,v in bigram_pmi.items() if v[0] >= 5}
list(bigram_pmi_filtered.items())[:10]

[(('świeckie', 'przygotowujące'), (5, 13.622991642182782)),
 (('ręcznego', 'miotacza'), (5, 13.622991642182782)),
 (('młyny', 'kulowe'), (5, 13.622991642182782)),
 (('zaszkodzić', 'wynikom'), (5, 13.622991642182782)),
 (('grzegorz', 'schetyna'), (5, 13.622991642182782)),
 (('mleczka', 'makowego'), (5, 13.440670085388827)),
 (('przeponowe', 'rurowe'), (5, 13.440670085388827)),
 (('adama', 'mickiewicza'), (6, 13.440670085388827)),
 (('papierem', 'wartościowym'), (5, 13.28651940556157)),
 (('chwytów', 'obezwładniających'), (5, 13.28651940556157))]

In [99]:
# zad 7
# constructing tree to speedup search
def construct_tree(n_grams):
    A__tree = {}
    _B_tree = {}
    for n_gram, freq in n_grams.items():
        t_first = n_gram[0]
        t_last = n_gram[-1]
        t_everything_but_first = n_gram[1:]
        #t_everything_but_last = n_gram[:-1]
        if len(n_gram) == 2:
            t_everything_but_first = t_everything_but_first[0]
            #t_everything_but_last = t_everything_but_last[0]
            
        if t_first not in A__tree:    
            A__tree[t_first] = {t_everything_but_first:freq}
        if t_everything_but_first not in A__tree[t_first]:
            A__tree[t_first][t_everything_but_first] = freq
        if t_everything_but_first not in _B_tree:    
            _B_tree[t_everything_but_first] = {t_first:freq}
        if t_first not in _B_tree[t_everything_but_first]:
            _B_tree[t_everything_but_first][t_first] = freq

    return A__tree, _B_tree 

In [100]:
A__tree, _B_tree = construct_tree(bigrams)

In [101]:
def H(vals, N):
    res = 0
    for val in vals:
        val /= N
        if not val == 0:
            res += val*log(val)
    return res

def LLR_n_gram(a, b, A__tree, _B_tree):
    k11 = A__tree[a][b]
    k12 = sum(_B_tree[b].values()) - k11
    k21 = sum(A__tree[a].values()) - k11
    k22 = total_bigrams_occurances - k11 - k12 - k21
    N = k11 + k12 + k21 + k22
    return 2 * N * (H([k11,k12,k21,k22], N) - H([k11+k12,k21+k22], N) - H([k11+k21,k12+k22], N))
    

In [111]:
LLR_bigrams  = {k: LLR_n_gram(k[0], k[1], A__tree, _B_tree) for k in bigrams.keys()}
bigram_top_10_LLR = list(LLR_bigrams.keys())[:10]

In [113]:
#zad 8
bigram_top_10_LLR = {k: v for k,v in sorted(LLR_bigrams.items(), key=lambda item: item[1], reverse=True)}
list(bigram_top_10_LLR.items())[:10]

[(('mowa', 'w'), 162056.43970866656),
 (('których', 'mowa'), 103860.60989464173),
 (('o', 'których'), 81588.40541168801),
 (('którym', 'mowa'), 67846.84419746423),
 (('dodaje', 'się'), 62515.67442178589),
 (('do', 'spraw'), 57437.651725128424),
 (('o', 'którym'), 53202.142924775384),
 (('i', 'nr'), 51618.30513495073),
 (('z', 'dnia'), 47165.64226194411),
 (('na', 'podstawie'), 45750.135696047284)]

In [104]:
#zas 9, 10
trigrams = {k: v for k,v in trigrams.items() if only_polish_letters(k[0]) and only_polish_letters(k[1]) and only_polish_letters(k[2])}
A__trigrams_tree, _B_trigrams_tree = construct_tree(trigrams)

In [105]:
LLR_trigrams  = {k: LLR_n_gram(k[0], k[1:], A__trigrams_tree, _B_trigrams_tree) for k in trigrams.keys()}
LLR_trigrams = {k: v for k,v in sorted(LLR_trigrams.items(), key=lambda item: item[1], reverse=True)}

In [106]:
trigram_top_10_LLR = list(LLR_trigrams.keys())[:10]
list(LLR_trigrams.items())[:10]

[(('których', 'mowa', 'w'), 102495.7504384144),
 (('o', 'których', 'mowa'), 92046.18649791507),
 (('którym', 'mowa', 'w'), 68036.21581825346),
 (('o', 'którym', 'mowa'), 62514.58015060082),
 (('minister', 'właściwy', 'do'), 48241.9150999556),
 (('właściwy', 'do', 'spraw'), 47786.77952740153),
 (('której', 'mowa', 'w'), 38775.47991226523),
 (('ustawie', 'z', 'dnia'), 38352.70893325243),
 (('o', 'której', 'mowa'), 36304.48245822556),
 (('zastępuje', 'się', 'wyrazami'), 31100.75623494178)]

In [107]:
trigram_pmi = {}
total_trigram_occurances = sum(trigrams.values())

for trigram, freq in trigrams.items():
    t1, t2, t3 = trigram
    p1 = single_grams[t1]/total_single_occurances
    p2 = single_grams[t2]/total_single_occurances
    p3 = single_grams[t3]/total_single_occurances
    p1p2p3 = freq/total_trigram_occurances
    pmi = log(p1p2p3/(p1 * p2 * p3))
    trigram_pmi[trigram] = (freq, pmi)

In [108]:
trigram_pmi = {k: v for k,v in sorted(trigram_pmi.items(), key=lambda item: item[1][1], reverse=True) if v[0] >= 5}

trigram_top_10_pmi = list(trigram_pmi.keys())[:10]
list(trigram_pmi.items())[:10]

[(('piłce', 'nożnej', 'uefa'), (9, 25.219350097730103)),
 (('profilem', 'zaufanym', 'epuap'), (8, 25.200992842211733)),
 (('religijne', 'uroczystości', 'pogrzebowe'), (6, 25.160407562096655)),
 (('finałowego', 'turnieju', 'mistrzostw'), (6, 25.142058423428455)),
 (('potwierdzonym', 'profilem', 'zaufanym'), (11, 24.88345780661027)),
 (('turnieju', 'mistrzostw', 'europy'), (6, 24.879694158960966)),
 (('grożącą', 'jemu', 'samemu'), (5, 24.27739161910737)),
 (('bankowemu', 'funduszowi', 'gwarancyjnemu'), (5, 24.210349967751682)),
 (('komunalne', 'osady', 'ściekowe'), (5, 24.179578309084928)),
 (('konfesyjne', 'nauczanie', 'religii'), (5, 24.15710545323287))]

In [109]:
# zad 11
top_10_bigrams = bigram_top_10_pmi + bigram_top_10_LLR

In [119]:
# key -> freq -> pmi -> LLR
{k: (bigram_pmi[k], LLR_bigrams[k]) for k in top_10_bigrams}

{('kołowe', 'jednoosiowe'): ((1, 15.232429554616882), 31.12486567648942),
 ('automatyki', 'grzewczej'): ((1, 15.232429554616882), 31.12486567648942),
 ('prefabrykatów', 'wnętrzowe'): ((1, 15.232429554616882), 31.12486567648942),
 ('gołe', 'aluminiowe'): ((1, 15.232429554616882), 31.12486567648942),
 ('polistyrenu', 'spienionego'): ((1, 15.232429554616882), 31.12486567648942),
 ('objaśnieniem', 'figur'): ((1, 15.232429554616882), 31.12486567648942),
 ('wkładzie', 'wnoszonym'): ((1, 15.232429554616882), 31.12486567648942),
 ('doktorem', 'habilitowanym'): ((1, 15.232429554616882), 31.12486567648942),
 ('naruszonymi', 'plombami'): ((1, 15.232429554616882), 31.12486567648942),
 ('zaniedbanej', 'wychowawczo'): ((1, 15.232429554616882), 31.12486567648942),
 ('mowa', 'w'): ((27728, 2.980853838524479), 162056.43970866656),
 ('których', 'mowa'): ((13019, 4.640397578633193), 103860.60989464173),
 ('o', 'których'): ((12609, 3.801201625359062), 81588.40541168801),
 ('którym', 'mowa'): ((8722, 4.659

In [120]:
top_10_triagrams = trigram_top_10_pmi + trigram_top_10_LLR

In [121]:
# key -> freq -> pmi -> LLR
{k: (trigram_pmi[k], LLR_trigrams[k]) for k in top_10_triagrams}

{('piłce', 'nożnej', 'uefa'): ((9, 25.219350097730103), 234.0720551123133),
 ('profilem', 'zaufanym', 'epuap'): ((8, 25.200992842211733),
  215.72783421000346),
 ('religijne', 'uroczystości', 'pogrzebowe'): ((6, 25.160407562096655),
  146.12663928095893),
 ('finałowego', 'turnieju', 'mistrzostw'): ((6, 25.142058423428455),
  165.24806621257213),
 ('potwierdzonym', 'profilem', 'zaufanym'): ((11, 24.88345780661027),
  278.4573757476447),
 ('turnieju', 'mistrzostw', 'europy'): ((6, 24.879694158960966),
  159.50643775653444),
 ('grożącą', 'jemu', 'samemu'): ((5, 24.27739161910737), 125.74743292941659),
 ('bankowemu', 'funduszowi', 'gwarancyjnemu'): ((5, 24.210349967751682),
  131.15416554098505),
 ('komunalne', 'osady', 'ściekowe'): ((5, 24.179578309084928),
  110.03825080790341),
 ('konfesyjne', 'nauczanie', 'religii'): ((5, 24.15710545323287),
  139.52993978117863),
 ('których', 'mowa', 'w'): ((12545, 7.608131662525183), 102495.7504384144),
 ('o', 'których', 'mowa'): ((11712, 8.673299353

In [ ]:
#zad 12
#Why do we have to filter the bigrams, rather than the token sequence?
Przykład: ponad 100 zł, filtrując sekwencję tokenów dostaniemy tokeny: ponad, złotych, z czego uzyskamy bigram (ponad zł), 
wyrywany z kontektu i bez sensu (bez uwzględnienia kolejności słów).
Natomiast jeśli najpierw podzielimy zdanie na uzyskamy 2 bigramy: (ponad 100), (100 zł), które potem zostaną odfiltrowane, 
nie uzyskamy efektu popisanego wyżej.

#Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
Obie metody znajdują inne wyrażenia. W przypadku trigramów PMI daje bardziej ciekawe resultaty, natomiast LLR znajduje znacznie 
popularniejsze wyrażenia (podobnie jak zwykła frequency lista). Rozważając bigarmy, tendencja jest ta sama. PMI zwrócił 
zaskakujące, rzadko występujące wyrażenia.
Ze względu na całkiem inne działanie trudno dokonać porównania. 

#What types of expressions are discovered by the methods.
LLR znajduje powrzechnie wsytępujące wyrażenia np. "o którym mowa", natomiast PMI znajduje utarte zwroty, nazwy własne 
(np. "profilem zaufanym epuap").

#Can you devise a different type of filtering that would yield better results?
Usunięcie przyimków 'w', 'na', 'do' i na odfiltrowanych bigramach użycie LLR mogłoby dać ciekawe resultaty.